In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='.env')
openai_api_key = os.getenv("OPENAI_API_KEY")

carro_template = """Você é um especialista em automóveis. Você tem amplo conhecimento sobre mecânica de carros, \
modelos e tecnologia automotiva. Você fornece respostas claras e úteis sobre carros.\
\
Pergunta:\
{query}"""

piada_template = """Você é um comediante,alegre e divertido. Dada uma pergunta crie uma piada sobre a pergunta \
\
Pergunta:\
{query}"""

patent_template = """Você é um assistente que responde perguntas sobre pedidos de patente.\
\
Pergunta:\
{query}"""

In [2]:
carro_questions = [
    "Qual a diferença entre um sedan e um SUV?",
    "Explique o funcionamento de um carro elétrico?",
    "Explique a diferença de um motor de dois tempos e de quatro tempos?",
]

piada_questions = [
    "O que o pato disse para a pata? R.: Vem Quá!",
    "Qual é a cidade brasileira que não tem táxi? R.: Uberlândia.",
    "Por que o rádio não pode ter filhos? R.: Porque ele é estéreo.",
]

patent_questions = [
    "Qual o andamento do pedido de patente PI9809080?",
    "A patente PI9908990 está em vigor?",
    "Quando foi depositada a patente 112012004567?",
]

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-3-small", dimensions=1536)

carro_question_embeddings = embeddings.embed_documents(carro_questions)
piada_question_embeddings = embeddings.embed_documents(piada_questions)
patent_question_embeddings = embeddings.embed_documents(patent_questions)

In [4]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    carro_similarity = cosine_similarity([query_embedding], carro_question_embeddings)[0]
    piada_similarity = cosine_similarity([query_embedding], piada_question_embeddings)[0]
    patent_similarity = cosine_similarity([query_embedding], patent_question_embeddings)[0]

    max_similarity = max(max(carro_similarity), max(piada_similarity), max(patent_similarity))

    if max_similarity == max(carro_similarity):
        print(f"Questão relativa a carros...")
        return PromptTemplate.from_template(carro_template)
    elif max_similarity == max(piada_similarity):
        print(f"Questão relativa a piadas")
        return PromptTemplate.from_template(piada_template)
    else:
        print(f"Questão relativa a patente")
        return PromptTemplate.from_template(patent_template)

query = "Qual o melhor modo de melhorar a durabilidade da bateria do meu carro?"
input_query = {"query": query}
prompt = prompt_router(input_query)

Questão relativa a carros...


In [5]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [6]:
chain.invoke(query)

Questão relativa a carros...


'Para melhorar a durabilidade da bateria do seu carro, siga estas dicas:\n\n1. Evite deixar dispositivos eletrônicos ligados quando o motor estiver desligado, pois isso pode descarregar a bateria mais rapidamente.\n2. Realize manutenções regulares no sistema elétrico do veículo, verificando se não há fios desgastados ou conexões soltas.\n3. Mantenha a bateria limpa e livre de corrosão, limpando os terminais com uma escova de metal e uma solução de água e bicarbonato de sódio.\n4. Evite dar partida no carro com muita frequência sem que o motor aqueça completamente, pois isso pode sobrecarregar a bateria.\n5. Se o carro ficar parado por um longo período de tempo, desconecte a bateria ou utilize um carregador de bateria para mantê-la carregada.\n\nSeguindo essas dicas, você pode prolongar a vida útil da bateria do seu carro.'

In [7]:
query = "qual o status do pedido de patente ?"
input_query = {"query": query}
chain.invoke(query)

Questão relativa a patente


'Para verificar o status de um pedido de patente, é necessário consultar o Instituto Nacional da Propriedade Industrial (INPI) ou o órgão responsável pela concessão de patentes no país em questão. Geralmente, é possível acompanhar o andamento do pedido de patente através de um número de protocolo fornecido no momento da solicitação. Recomendo entrar em contato diretamente com o órgão competente para obter informações atualizadas sobre o status do pedido de patente.'

In [8]:
query = "conte-me uma piada de girafas ?"
input_query = {"query": query}
chain.invoke(query)

Questão relativa a piadas


'Claro! Por que as girafas têm pescoços tão compridos? Para conseguir dar beijos de bom dia nas nuvens!'